In [9]:
# --- Knowledge Dashboard (CSV-powered for Binder/Voila) ---

import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, HTML

# --- Step 1: Load pre-fetched dataset (no DB connection) ---
df = pd.read_csv("dashboard_data.csv")

# --- Step 2: Validate and clean ---
if df is None or len(df) == 0:
    print("No data returned for this query.")
else:
    # Convert and clean columns
    df['question_id'] = df['question_id'].astype(str)
    df['answer_id'] = df['answer_id'].astype(str)
    df['grade_level'] = df['grade_level'].fillna('Unspecified').astype(str)
    df['academic_year_id'] = df['academic_year_id'].fillna(0).astype(int)
    df['avg_knowledge_pct'] = df['avg_knowledge_pct'].fillna(0)
    df['student_count'] = df['student_count'].fillna(0)

    # --- Map academic years ---
    year_labels = {
        2: '2021–2022',
        3: '2022–2023',
        4: '2023–2024',
        5: '2024–2025',
        0: 'Unspecified'
    }
    df['academic_year_label'] = df['academic_year_id'].map(year_labels).fillna('Unspecified')

    # --- Dropdown widgets ---
    dropdown_year = widgets.Dropdown(
        options=sorted([y for y in df['academic_year_label'].unique() if y != 'Unspecified'], reverse=True),
        value='2024–2025',
        description='Academic Year:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='45%')
    )

    dropdown_grade = widgets.Dropdown(
        options=sorted(df['grade_level'].unique().tolist()),
        value=sorted(df['grade_level'].unique().tolist())[0],
        description='Grade Level:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='30%')
    )

    # --- Function to populate questions based on year & grade ---
    def build_question_options(grade, year):
        subset = df[(df['grade_level'] == grade) & (df['academic_year_label'] == year)]
        questions = (
            subset[['question_id', 'question_text']]
            .drop_duplicates()
            .sort_values('question_text')
        )

        def truncate(text, n=90):
            return text if len(text) <= n else text[:n-3] + "..."

        return [(truncate(qt), qid) for qid, qt in zip(questions['question_id'], questions['question_text'])]

    question_dropdown = widgets.Dropdown(
        description='Question:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='90%')
    )
    question_dropdown.options = build_question_options(dropdown_grade.value, dropdown_year.value)
    question_dropdown.value = question_dropdown.options[0][1]

    # --- Output area for chart ---
    output_chart = widgets.Output()

    # --- Chart update logic ---
    def update_chart(*args):
        global fig
        with output_chart:
            output_chart.clear_output(wait=True)
            grade = dropdown_grade.value
            year = dropdown_year.value
            question_id = question_dropdown.value

            filtered = df[
                (df['grade_level'] == grade) &
                (df['academic_year_label'] == year) &
                (df['question_id'] == question_id)
            ]

            if filtered.empty:
                display(HTML("<p style='color:red;'>No data available for this selection.</p>"))
                return

            q_text = filtered['question_text'].iloc[0]
            display(HTML(f"<h4 style='margin-top:10px;'>{q_text}</h4>"))

            filtered = filtered.sort_values('avg_knowledge_pct', ascending=False)
            filtered['status'] = filtered['student_count'].apply(lambda x: 'Answered' if x > 0 else 'No Data')

            fig = px.bar(
                filtered,
                x='avg_knowledge_pct',
                y='answer_text',
                orientation='h',
                color='status',
                color_discrete_map={'Answered': '#4CAF50', 'No Data': '#CCCCCC'},
                text='avg_knowledge_pct',
                hover_data={
                    'answer_id': True,
                    'student_count': True,
                    'avg_knowledge_pct': ':.1f',
                    'answer_text': True
                },
                labels={'avg_knowledge_pct': 'Avg Knowledge %', 'answer_text': 'Answer'},
                height=550
            )

            fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
            fig.update_layout(
                title=f"Grade {grade} — {year}: Average Knowledge % by Answer",
                xaxis=dict(title='Average Knowledge %', range=[0, 100]),
                yaxis=dict(title='', automargin=True),
                plot_bgcolor='white',
                margin=dict(l=150, r=30, t=50, b=40),
                legend_title_text='Response Status'
            )

            fig.show()

    # --- Update question dropdown when filters change ---
    def on_filter_change(change):
        new_grade = dropdown_grade.value
        new_year = dropdown_year.value
        new_options = build_question_options(new_grade, new_year)
        if new_options:
            question_dropdown.options = new_options
            question_dropdown.value = new_options[0][1]
            update_chart()

    dropdown_year.observe(on_filter_change, names='value')
    dropdown_grade.observe(on_filter_change, names='value')
    question_dropdown.observe(update_chart, names='value')

    # --- Display the full dashboard ---
    ui = widgets.VBox([
        widgets.HBox([dropdown_year, dropdown_grade]),
        question_dropdown,
        output_chart
    ])

    display(ui)
    update_chart()
